In [1]:
import os
from data_ml_utils.pyathena_client.client import PyAthenaClient

os.environ["AWS_ACCESS_KEY_ID"] = "xxx"
os.environ["AWS_SECRET_ACCESS_KEY"] = "xxx"

### initialise pyathena client

In [2]:
pyathena_client = PyAthenaClient()

### query with `pyathena_client`
- the way we are querying with athena, saves `int64` as pandas `Int64`
    - this data type is incompatible with most of the pandas function we will use
    - we will have to convert them to 
- almost one-liner to query results and return as pandas dataframe

In [3]:
query = """
SELECT
    *
FROM
    dev.test_tutorial_table
LIMIT 10
"""

df_raw = pyathena_client.query_as_pandas(final_query=query)
df_raw.dtypes

dummy_column_one     int64
dummy_column_two     int64
date_created        object
dtype: object

In [4]:
df_raw.head()

,dummy_column_one,dummy_column_two,date_created
0,1,11,2022-01-01
1,2,12,2022-01-01
2,3,13,2022-01-01
3,4,14,2022-01-01
4,5,15,2022-01-01


### drop athena table
- we will use `dev.test_tutorial_table` as an example
- one-liner to drop athena table
- if successful returns 0, else 1

In [5]:
pyathena_client.drop_table(
    table_name="test_tutorial_table",
    database="dev"
)

0

### creating table with schema (or updating partitions of table) in offline feature store
- if there are changes to schema, table has to be dropped and recreated again
- we will use `dev.test_tutorial_table` as an example
- if successful returns 0, else 1

- one-liner to create table and update partitions

- in athena, we have to manually add the newly created (or missing partitions) data by running `msck repair table` SQL command
    - `msck repair table` will look at all the partitions within the s3 directory, and add partitions that are missing

In [6]:
pyathena_client.create_msck_repair_table(
    create_raw_query="tutorial_sql_schema/create_table_schema.sql",
    repair_raw_query="tutorial_sql_schema/msck_repair_table.sql",
    yaml_schema_file_path="tutorial_sql_schema/test_tutorial_table.yaml"
)

0